# Notebook to transform labels in a Yolo configuration


## Import useful modules


In [1]:
# General modules
import os
import subprocess
import shutil
import random

# Data manipulation modules
import pandas as pd
import json
from sklearn.model_selection import train_test_split

# Image and video modules
from PIL import Image
from plotly import express as px
import cv2

# Machine learning module
from ultralytics import YOLO

/Users/stephane_branly/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Launch `label-studio` to label images


In [ ]:
os.environ["LABEL_STUDIO_LOCAL_FILES_SERVING_ENABLED"] = "true"
os.environ["LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT"] = os.getcwd()

subprocess.run("label-studio")

## Read `label-studio` labels file and available classes


In [9]:
with open("./assets/labels.json", encoding="utf-8") as f:
    annotations = json.load(f)

classes = pd.read_csv("./assets/classes.csv", names=["_"])["_"].to_list()

## Display dataset images


In [10]:
def generate_image(annotation):
    path = "/".join(annotation["image"].split("/")[-2:])
    path = path.replace("?d=", "")
    path = path.replace("%28", "(")
    path = path.replace("%29", ")")
    path = path.replace("?d=", "")
    fig = px.imshow(Image.open(path))
    for label in annotation.get("label", []):
        x = round(label["x"] / 100 * label["original_width"])
        y = round(label["y"] / 100 * label["original_height"])
        width = round(label["width"] / 100 * label["original_width"])
        height = round(label["height"] / 100 * label["original_height"])
        str_label = label["rectanglelabels"][0]
        fig.add_shape(
            type="rect",
            x0=x,
            x1=x + width,
            y0=y,
            y1=y + height,
            xref="x",
            yref="y",
            line_color="red",
        )
        fig.add_annotation(
            x=x,
            y=y,
            text=str_label,
            xref="x",
            xanchor="left",
            yref="y",
            yanchor="bottom",
            showarrow=False,
            font_color="white",
            bgcolor="red",
        )
    return fig

_Run the next cell to explore randomly your dataset_


In [72]:
fig = generate_image(random.choice(annotations))

fig.update_layout(margin=dict(l=20, r=20, t=20, b=20), width=640, height=500)

## Check the label distribution


In [84]:
count = {}
for annotation in annotations:
    for l in annotation.get("label", []):
        k = l["rectanglelabels"][0]
        count[k] = count.get(k, 0) + 1

In [85]:
fig = px.bar(
    x=classes,
    y=[count[c] for c in classes],
    color=[c.split(" ")[0] for c in classes],
)
fig.update_layout(
    xaxis={"dtick": classes},
    xaxis_title=None,
    yaxis_title="Annotation count",
    legend_title="Suit",
)

## Generate Yolo annotation file


In [11]:
def generate_yolo_annotations(annotations):
    for a in annotations:
        path = a["image"].split("/")[-1]
        path = path.replace("?d=", "")
        path = path.replace("%28", "(")
        path = path.replace("%29", ")")
        path = path.replace("?d=", "")
        with open(f"labels/{path[:-3]}txt", mode="w", encoding="utf8") as f:
            for l in a.get("label", []):
                x_min = l["x"] / 100
                y_min = l["y"] / 100
                box_width = l["width"] / 100
                box_height = l["height"] / 100
                x_center = x_min + box_width / 2
                y_center = y_min + box_height / 2
                label = classes.index(l["rectanglelabels"][0])
                f.write(
                    f"{label} {x_center:.3f} {y_center:.3f} {box_width:.3f} {box_height:.3f}\n"
                )

In [18]:
!mkdir labels

In [19]:
generate_yolo_annotations(annotations)

## Split dataset into train, valid and test dataset


In [20]:
# Read images and annotations
images = [
    os.path.join("dataset", x)
    for x in os.listdir("dataset")
    if x[-3:] == "jpg"
    if x[:-3] in [y[:-3] for y in os.listdir("labels")]
]
labels = [os.path.join("labels", x) for x in os.listdir("labels") if x[-3:] == "txt"]

images.sort()
labels.sort()

# Split the dataset into train-valid-test splits
train_images, val_images, train_labels, val_labels = train_test_split(
    images, labels, test_size=0.2, random_state=1
)
val_images, test_images, val_labels, test_labels = train_test_split(
    val_images, val_labels, test_size=0.5, random_state=1
)

### Create directories to move images and labels


In [21]:
!mkdir images images/train images/val images/test labels/train labels/val labels/test

### Move files in directories


In [22]:
# Utility function to move images
def move_files_to_folder(list_of_files, destination_folder, move=False):
    for f in list_of_files:
        try:
            if move:
                shutil.move(f, destination_folder)
            else:
                shutil.copy(f, destination_folder)
        except:
            print(f)
            assert False


# Copy the images into their folders
move_files_to_folder(train_images, "images/train")
move_files_to_folder(val_images, "images/val/")
move_files_to_folder(test_images, "images/test/")
# Move the labels into their folders
move_files_to_folder(train_labels, "labels/train/", move=True)
move_files_to_folder(val_labels, "labels/val/", move=True)
move_files_to_folder(test_labels, "labels/test/", move=True)

# Train


In [7]:
import os

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
model = YOLO("runs/detect/train6/weights/last.pt")
# model = YOLO("yolov8n.pt")

model.train(
    data="tarot_card_data.yaml", epochs=100, imgsz=640, device="mps", resume=True
)

## Valid


In [2]:
# Load a model
from ultralytics import YOLO

model = YOLO("./runs/detect/train6/weights/last.pt")

# Customize validation settings
validation_results = model.val(
    data="tarot_card_data.yaml", imgsz=640, batch=16, conf=0.5, iou=0.7, device="mps"
)

Ultralytics YOLOv8.1.34 🚀 Python-3.9.6 torch-2.2.1 MPS (Apple M3 Pro)
Model summary (fused): 168 layers, 3134342 parameters, 0 gradients, 8.7 GFLOPs


val: Scanning /Users/stephane_branly/Documents/Projets/FrenchTarot/card-recognition-model/labels/val.cache... 122 images, 0 backgrounds, 0 corrupt: 100%|██████████| 122/122 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.17it/s]


                   all        122        163      0.823      0.731      0.814      0.776
           diamond ace        122          2        0.5        0.5      0.622       0.56
           diamond two        122          2          1        0.5       0.75      0.675
         diamond three        122          2          1        0.5       0.75       0.75
          diamond four        122          3          1      0.667      0.833      0.833
          diamond five        122          1        0.5          1      0.995      0.895
           diamond six        122          3          1          1      0.995      0.842
         diamond seven        122          1        0.5          1      0.995      0.995
         diamond eight        122          3          1          1      0.995      0.995
           diamond ten        122          1          1          1      0.995      0.995
        diamond knight        122          5          1        0.6        0.8      0.699
         diamond quee

## Track


In [ ]:
# Load the YOLOv8 model
model = YOLO("./runs/detect/train6/weights/last.pt")

# Open the video file
video_path = "http://192.168.1.249:4747/video"
cap = cv2.VideoCapture(video_path)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, show=False)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

## Convert to tensorflow light app


In [10]:
model = YOLO("./runs/detect/train5/weights/last.pt")

In [12]:
model_onnx = model.export(format="onnx")

import onnx2tf

tf_model = onnx2tf.convert(model_onnx, output_tfv1_pb=True)

Ultralytics YOLOv8.1.34 🚀 Python-3.9.6 torch-2.2.1 CPU (Apple M3 Pro)
Model summary (fused): 168 layers, 3134342 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'runs/detect/train5/weights/last.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 82, 8400) (36.5 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 0.5s, saved as 'runs/detect/train5/weights/last.onnx' (12.2 MB)

Export complete (0.7s)
Results saved to /Users/stephane_branly/Documents/Projets/FrenchTarot/card-recognition-model/runs/detect/train5/weights
Predict:         yolo predict task=detect model=runs/detect/train5/weights/last.onnx imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train5/weights/last.onnx imgsz=640 data=tarot_card_data.yaml  
Visualize:       https://netron.app


In [1]:
pip install onnx2tf --upgrade
import onnx2tf

Defaulting to user installation because normal site-packages is not writeable
  Using cached onnx2tf-1.20.0-py3-none-any.whl.metadata (130 kB)
Using cached onnx2tf-1.20.0-py3-none-any.whl (412 kB)
  Attempting uninstall: onnx2tf
    Found existing installation: onnx2tf 1.17.5
    Uninstalling onnx2tf-1.17.5:
      Successfully uninstalled onnx2tf-1.17.5
Note: you may need to restart the kernel to use updated packages.


In [14]:
tf_model = onnx2tf.convert(model_onnx, output_tfv1_pb=True)


Model optimizing started ============================================================
Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃            ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add        │ 9              │ 8                │
│ Concat     │ 19             │ 19               │
│ Constant   │ 149            │ 143              │
│ Conv       │ 64             │ 64               │
│ Div        │ 2              │ 1                │
│ Gather     │ 1              │ 0                │
│ MaxPool    │ 3              │ 3                │
│ Mul        │ 60             │ 58               │
│ Reshape    │ 5              │ 5                │
│ Resize     │ 2              │ 2                │
│ Shape      │ 1              │ 0                │
│ Sigmoid    │ 58             │ 58               │
│ Slice      │ 2              │ 2                │
│ Softmax    │ 1              │ 1                │

W0000 00:00:1713384140.239950 2133528 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1713384140.239959 2133528 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.


Float32 tflite output complete!
Float16 tflite output complete!


W0000 00:00:1713384140.665673 2133528 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1713384140.665683 2133528 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
